# Classification - Decision Tree

In [2]:
# pip install pydotplus

In [3]:
 #pip install graphviz

If the pip install command is not enough follow the next steps:
- In "Anaconda Prompt" execute the command "conda install graphviz" 
- Add to the PATH: C:\Users\username\Anaconda3\Library\bin\graphviz. To modify PATH goto: Control Panel > System and Security > System > Advanced System Settings > Environment Variables > Path > Edit > New
- Restart your computer

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics  
from sklearn.model_selection import KFold
from sklearn.tree import export_graphviz
from IPython.display import Image  
import pydotplus

In [5]:
df = pd.read_csv("./data/Titanic_final.csv", index_col=0)
df.head()

,Survived,Pclass,SibSp,Parch,Sex_cat,Embarked_cat,Title_cat,Is_Alone,Fare_min_max,Age_norm
0,0,3,1,0,1,2,2,0,0.014151,-0.517838
1,1,1,1,0,0,0,3,0,0.139129,0.615799
2,1,3,0,0,0,2,1,1,0.015478,-0.234429
3,1,1,1,0,0,2,3,0,0.103644,0.403242
4,0,3,0,0,1,2,2,1,0.015713,0.403242


Split the data set to X and y.<br>
X for the features and y for the target. 

In [6]:
X = df.drop('Survived', axis=1)
y = df['Survived']

To evaluate the model we split the data set to train set and test set (= validation).<br>
We train the model <b> only </b> on the train set and evaluate it on the test set.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) 

Create Decision Tree classifier object, train it on the train set and predict the labels to the test set.<br>

In [8]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(random_state=42) 

# Train Decision Tree Classifer 
clf = clf.fit(X_train,y_train)

# Predict the label
y_pred = clf.predict(X_test)

Note: sklearn uses an optimised version of the CART algorithm.

#### Acuuracy and confusion matrix:

In [9]:
print(metrics.accuracy_score(y_test, y_pred))
pd.DataFrame(metrics.confusion_matrix(y_test, y_pred))


0.7443946188340808


,0,1
0,103,31
1,26,63


### Decision tree visualization
One of the decision trees advantages is the ability to visualize the decision rules and therefore they are easy to understand and interpret. 

In [11]:
dot = export_graphviz(clf, filled=True, rounded=True, 
                special_characters=True,feature_names = X.columns,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot)  
graph.write_png('tree1.png')
Image(graph.create_png())

InvocationException: GraphViz's executables not found

#### Predict the labels for the train set

In [ ]:
y_train_pred = clf.predict(X_train)
print(metrics.accuracy_score(y_train, y_train_pred))
pd.DataFrame(metrics.confusion_matrix(y_train, y_train_pred))

## Overfitting!!
Changing hyper-parameters effects the complexity of the model. Too complex (=deep) model will overfit, too simple (=shallow) model will underfit.<br>
Configure a better model by limits the size of the tree.

In [ ]:
clf2 = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=8,
                              min_samples_split=15, min_samples_leaf=5, random_state=42) 

clf2 = clf2.fit(X_train,y_train)

In [ ]:
dot = export_graphviz(clf2, filled=True, rounded=True, 
                special_characters=True,feature_names = X.columns,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot)  
graph.write_png('tree2.png')
Image(graph.create_png())

#### train

In [ ]:
y_train_pred2 = clf2.predict(X_train)
print(metrics.accuracy_score(y_train, y_train_pred2))
pd.DataFrame(metrics.confusion_matrix(y_train, y_train_pred2))

#### test

In [ ]:
y_pred2 = clf2.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred2))
pd.DataFrame(metrics.confusion_matrix(y_test, y_pred2))

Let's see how a single parameter can affect the performances, in the <b> train </b> and in the <b> test </b>:

In [ ]:
train_scores, test_scores = [],[]
for k in range(2,50):
    clf3 = DecisionTreeClassifier(criterion='entropy', splitter='best', 
                              max_depth=k, random_state=42) 
    clf3.fit(X_train,y_train)
    train_scores.append(clf3.score(X_train, y_train))
    test_scores.append(clf3.score(X_test, y_test))
plt.plot(range(2,50), train_scores)
plt.plot(range(2,50), test_scores)
plt.legend(['train','test'])
plt.show()

### Changing the splits to be by category

In [ ]:
X.head()

In [ ]:
for col in X:
    print(col, len(X[col].unique()))

In [ ]:
dummies_col = [col for col in df if len(df[col].unique()) > 2 and len(df[col].unique()) < 8]
X_dummies = pd.get_dummies(data = X, columns = dummies_col )
X_dummies.head()

In [ ]:
X_train_d, X_test_d, y_train, y_test = train_test_split(X_dummies, y, test_size=0.25, random_state=42) 

In [ ]:
clf3 = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=8,
                              min_samples_split=15, min_samples_leaf=5, random_state=42) 

clf3 = clf3.fit(X_train_d,y_train)

In [ ]:
y_pred3 = clf3.predict(X_test_d)
print(metrics.accuracy_score(y_test, y_pred3))
pd.DataFrame(metrics.confusion_matrix(y_test, y_pred3))

In [ ]:
dot = export_graphviz(clf3, filled=True, rounded=True, 
                special_characters=True,feature_names = X_dummies.columns,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot)  
graph.write_png('tree3.png')
Image(graph.create_png())

## Model evaluation
Comparison between two models by using cross-validation to get average results.

In [ ]:
clf_a = DecisionTreeClassifier(criterion='entropy', splitter='best', 
                          max_depth=8, min_samples_split=15, min_samples_leaf=5, random_state=42)
clf_b = DecisionTreeClassifier(criterion='entropy', splitter='best', 
                          max_depth=8, min_samples_split=15, min_samples_leaf=5, random_state=42) 
X = X.to_numpy()
X_dummies = X_dummies.to_numpy()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cm_a, cm_b = [],[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    X_train_d, X_test_d = X_dummies[train_index], X_dummies[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf_a.fit(X_train,y_train)
    clf_b.fit(X_train_d,y_train)
    y_pred_a = clf_a.predict(X_test)
    y_pred_b = clf_b.predict(X_test_d)
    cm_a.append(metrics.confusion_matrix(y_test, y_pred_a))
    cm_b.append(metrics.confusion_matrix(y_test, y_pred_b))

Calculate mean accuracy for each of the models.

In [ ]:
acc_a = [(cm[0][0]+cm[1][1])/sum(sum(cm)) for cm in cm_a]
np.mean(acc_a)

In [ ]:
acc_b = [(cm[0][0]+cm[1][1])/sum(sum(cm)) for cm in cm_b]
np.mean(acc_b)

### What is the evaluation metric we should use in this problem?
Confusion matrix: 

|  | predicted negative | predicted positive |
|:--- | :---: | :---: |
| actual negative | TN | FP|
| actual positive | FN | TP|

- sensitivity = recall =  $ \frac{TP}{TP+FN} $ 
- specificity = $ \frac{TN}{TN +FP} $
- precision = $ \frac{TP}{TP+FP}$



In [ ]:
recall_a = [(cm[1][1]/(cm[1][1]+cm[0][1])) for cm in cm_a]
#recall_a
np.mean(recall_a)

In [ ]:
recall_b = [(cm[1][1]/(cm[1][1]+cm[0][1])) for cm in cm_b]
#recall_b
np.mean(recall_b)